## FACE RECOGNITION

In [ ]:
# for image scraping from webpages
from bs4 import BeautifulSoup
# for handling URLs
from urllib.request import urlopen
# for regular expression
import re
# for interacting with the operating system
import os
# for sending HTTP requests
import requests as req
# for reconginizing faces in a photograph or video frame
import face_recognition
# for video and image processing
import cv2
# for matching filepaths
import glob
# for working with arrays 
import numpy as np
# for formatting date objects into readable strings
import datetime
# to copy the screen sontent to a PIL image memory
from PIL import ImageGrab
# to retrieve a specified system configuration settings for height and width
from win32api import GetSystemMetrics
# selecting random values for the colour of the text output
from random import randrange

from tkinter import Tk     
from tkinter.filedialog import askopenfilename


SELECT YOUR OPTION ACCORDING TO THE TASK TO BE PERFORMED 
1. Face recognition on a Videofile / screen recording
2. Face recognition in real time
3. Exit

In [ ]:
counter = int(input())

DO YOU WANT TO CREATE A NEW DATASET TO INCLUDE FOR ABOVE TASKS ?

1. YES
2. NO

In [ ]:
dataset_counter = int(input())

For adding new images to recognize just add the .jpg files to the images folder.
I have written the code for bulk adding photos as well just add the full names of the people to add_names.txt and their images will automatically be downloaded to the images folder.
Basically we are doing web scraping from wikipedia pages to get the images of the people.
In this instance we have taken several academy award winners and a lot of bollywood actors.

In [ ]:
if dataset_counter == 1:
  # giving the path of the text file to be read
  data_file = open('add_names.txt', "r")
  # reading the file line by line
  file_content = data_file.read()
  # replacing whitespaces with '_' 
  file_content = file_content.replace(" ", '_')
  # storing the contents of the text file into a list format
  content_list = file_content.split("\n")
  print(content_list)
  data_file.close()

  #creating a directory named images 
  try:
    os.mkdir('images')
  except:
    pass

  # looping through the entire list
  for i in range(0, len(content_list)):
      try:
        # providing the url to be accessed
        website_url = 'https://en.wikipedia.org/wiki/' + content_list[i]
        htmldata = urlopen(website_url)
        soup = BeautifulSoup(htmldata, 'html.parser')
        # finding images from the webpage with the extension '.jpg' only
        images = soup.find_all('img', {'src': re.compile('.jpg')})
        # getting the first '.jpg' image available on the webpage
        img_data = req.get('https:' + images[0]['src']).content
        # downloading and storing the images in the images directory created earlier
        with open('images/' + content_list[i] + '.jpg', 'wb+') as f:
          f.write(img_data)
      except:
          pass



FOR SCREEN RECORDIDING USE SCREEN_REC.PY

In [ ]:


# # Screen Recording Part 
# if(counter == 1):
#     print("Use the screen_rec.py or any other screen recording software to get the source file for face recognition ")
#     Tk().withdraw()  # we don't want a full GUI, so keep the root window from appearing
#     # show an "Open" dialog box and return the path to the selected file
#     file_name = askopenfilename()
#     print(file_name)

#     # # getting the height and width of the screen from the system
#     # width = GetSystemMetrics(0)
#     # height = GetSystemMetrics(1)
#     # #get the current time of the scren recording and convert it into a string
#     # time_stamp = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
#     # # format the name of the screen recording file to be saved
#     # screen_recording = f'{time_stamp}.mp4'
#     # # giving characters to perform encoding operations
#     # enc_chars = cv2.VideoWriter_enc_chars('m', 'p', '4', 'v')
#     # vid_captured = cv2.VideoWriter(screen_recording, enc_chars, 20.0, (width, height))

#     # while True:
#     #     # grab the image from the screen
#     #     img = ImageGrab.grab(bbox=(0, 0, width, height))
#     #     # convert the image into a numpy array 
#     #     img_arr = np.array(img)
#     #     #convert the colour of the image to an RGB colour
#     #     final_img = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
#     #     #display the image
#     #     cv2.imshow('Screen Recording', final_img)

#     #     # write the final image captured
#     #     vid_captured.write(final_img)
#     #     # to stop screen recording
#     #     if cv2.waitKey(10) == ord('q'):
#     #         break
#     # cv2.destroyAllWindows()


In [ ]:
class SimpleFaceRecognition:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []

        # Resize frame for a faster speed
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        # Load Images
        images_path = glob.glob(os.path.join(images_path, "*.*"))

        print("{} encoding images found.".format(len(images_path)))

        # Store image encoding and names
        for img_path in images_path:
            try:
              img = cv2.imread(img_path)
              rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

              # Get the filename only from the initial file path.
              basename = os.path.basename(img_path)
              (filename, ext) = os.path.splitext(basename)
              # Get encoding
              img_encoding = face_recognition.face_encodings(rgb_img)[0]

              # Store file name and file encoding
              self.known_face_encodings.append(img_encoding)
              self.known_face_names.append(filename)
            except:
              pass
        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        try:
          small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        except:
          return
        # Find all the faces and face encodings in the current frame of video
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = self.known_face_names[best_match_index]
            face_names.append(name)

        # Convert to numpy array to adjust coordinates with frame resizing quickly
        face_locations = np.array(face_locations)
        face_locations = face_locations / self.frame_resizing
        return face_locations.astype(int), face_names


In [54]:
# Encode faces from a folder
SimpleFaceRecognition_obj = SimpleFaceRecognition()
SimpleFaceRecognition_obj.load_encoding_images("images/")

In [ ]:
if(counter == 1):
    print("Use the screen_rec.py or any other screen recording software to get the source file for face recognition ")
    Tk().withdraw()  # we don't need a full GUI, so we keep the root window from appearing
    # show an "Open" dialog box and return the path to the selected file
    file_name = askopenfilename()
    print(file_name)


In [ ]:
if counter == 1:
    cap = cv2.VideoCapture(file_name)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    f = open("screen_rec_output.avi", 'w')
    f.close()
    out = cv2.VideoWriter('screen_rec_output.avi', cv2.VideoWriter_fourcc(
        'M', 'J', 'P', 'G'), 10, (frame_width, frame_height))
elif counter == 2:
    cap = cv2.VideoCapture(0)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    f = open("real_time_output_video.avi", 'w')
    f.close()
    out = cv2.VideoWriter('real_time_output_video.avi', cv2.VideoWriter_fourcc(
        'M', 'J', 'P', 'G'), 10, (frame_width, frame_height))


names_set = []
while True:
    ret, frame = cap.read()

    # Detect Faces
    if(SimpleFaceRecognition_obj.detect_known_faces(frame) == None):
      break
    face_locations, face_names = SimpleFaceRecognition_obj.detect_known_faces(
        frame)
    for face_loc, name in zip(face_locations, face_names):
        y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
        red = randrange(128, 256)
        green = randrange(128, 256)
        blue = randrange(128, 256)
        cv2.putText(frame, name, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_DUPLEX, 1, (red, green, blue), 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (red, green, blue), 4)
        names_set.append(name)
        #print(name)

    cv2.imshow("Frame",frame)      
    out.write(frame)
    key = cv2.waitKey(1)
    if key == 27:
        break



res = []
for i in names_set:
    if i not in res:
        res.append(i)

print(res)

cap.release()
cv2.destroyAllWindows()


In [ ]:
if(counter == 1 ):
    print("The program has been executed successfully you face labelled file is saved as screen_rec_output.avi")
if(counter == 2):
    print("The program has been executed successfully you face labelled file is saved as real_time_output_video.avi")
